In [1]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from torch.utils.data import Dataset
from mingpt.utils import data_cleaning, CharDataset
from mingpt.utils import sample

In [3]:
model = torch.load('./saved_model.pth')
# print(model)

In [4]:
block_size = 6 # spatial extent of the model for its context
text = open('data/hg38.500m.100m.10m.ab', 'r').read() 
train_dataset = CharDataset(data_cleaning(text), block_size) 

============data cleaning================
characters before data cleaning:  {'\n', 'T', 'G', 'a', 't', 'N', 'g', 'c', 'C', 'A'}
characters after data cleaning:  {'T', 'G', 'N', 'C', 'A'}
data has 10280157 characters, 5 unique.


In [5]:
from mingpt.trainer import Trainer, TrainerConfig

batch_size = 2048 * 4

## initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=1, batch_size=batch_size, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=24)
trainer = Trainer(model, train_dataset, None, tconf)

In [6]:
context = "AAANAA"
x = torch.tensor([train_dataset.stoi[s] for s in context],
                  dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 100, temperature=1, sample=True, top_k=5)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

AAANAAAAAACACACACACAGCATCTGTAGGATTGCTGGTTGTTTCTGTTTGTGAGAAGCAAATAGTGACAATGAGCTTTGAAGGCGGGGGGAAGATTACTGAGCG
